# Feed Forwad Neural network

In [2]:
import os
import sys
import tqdm
import torch
import numpy as np
import torch.nn as nn
import seaborn as sns
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
from torchvision.datasets import FashionMNIST

sns.set()

print(f'tqdm version is  :{tqdm.__version__}')
print(f'tqdm version is  :{torch.__version__}')
print(f'numpy version is  :{np.__version__}')
print(f'tqdm version is  :{sns.__version__}')

tqdm version is  :4.64.0
tqdm version is  :1.11.0+cpu
numpy version is  :1.21.5
tqdm version is  :0.11.2


In [4]:
# Get Dataset
train_dataset = FashionMNIST(
    root='data',
    train=True,
    transform=ToTensor(),
    download=False
)

test_dataset = FashionMNIST(
    root='data',
    train=False,
    transform=ToTensor(),
    download=False
)

In [15]:
# Set the parameters
epochs = 5
batch_size = 100
num_classes = 10
hidden_size = 500
input_size = 28 * 28
learning_rate = 0.01

In [10]:
# Defining data loading pipeline 
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [28]:
# Let's define the model 
from turtle import forward


class NeuralNet(nn.Module):
    # Define the class attributes
    def __init__(self, input_size = 784, hidden_size=500, num_classes=10):
        super(NeuralNet, self).__init__()
        # Setting attributes values 
        self.input_size = input_size
        self.num_classes = num_classes
        self.hidden_size = hidden_size

        self.fc1 = nn.Linear(in_features=self.input_size, 
                            out_features=self.hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(in_features=self.hidden_size,
                             out_features=self.num_classes)
    
    # Define forwars pass
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [29]:
device = 'gpu' if torch.cuda.is_available() else 'cpu'

# initialize the model 
model = NeuralNet(
    input_size=input_size,
    hidden_size=hidden_size,
    num_classes=num_classes
).to(device)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [32]:
# Let's deinfe the training loop
total_steps = len(train_dataset)
print(f'total steps are : {total_steps}')
for epoch in tqdm.tqdm(range(epochs)):
    for i, (images, labels) in enumerate(train_loader):
        # We need to reshape the images 
        images = images.reshape(-1, input_size)
        images = images.to(device)
        labels = labels.to(device)
        # Let's define the forward loop
        outputs = model(images)
        l = loss(outputs, labels)

        # Let's deifne the backpropagation and optimization 
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print(f'epoch is : {epoch}, loss is : {l.item()}')
        


total steps are : 60000


 20%|██        | 1/5 [00:13<00:53, 13.48s/it]

epoch is : 0, loss is : 0.6293003559112549


 40%|████      | 2/5 [00:26<00:40, 13.43s/it]

epoch is : 1, loss is : 0.5525667071342468


...

epoch is : 2, loss is : 0.5119879245758057


 80%|████████  | 4/5 [00:53<00:13, 13.35s/it]

epoch is : 3, loss is : 0.48824813961982727


100%|██████████| 5/5 [01:06<00:00, 13.38s/it]

epoch is : 4, loss is : 0.4728589355945587


In [33]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 81.72 %
